In [15]:
import os
import re
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from sklearn.utils import shuffle

def getDataFromFiles(filefolder, category1 = "*", category2= "*", error_del = False, min_len=100, drop_duplicates=False, maxrow_by_category = 0) :
    if category1 == "*" : 
        category1 = set([re.search('_.*_', str(filename)).group().replace("_","") for filename in Path(filefolder).rglob("*.json")])
    else : 
        category1 = category1.split(",")
        
    if category2 == "*" : 
        category2 = set([re.search('_.*\.json', str(filename)).group().replace(".json","").split("_")[2] for filename in Path(filefolder).rglob("*.json")])
    else : 
        category2 = category2.split(",")
    
    df_li = []
    for cate1 in category1 :
        for cate2 in category2 :
            pattern = "*_{}_{}.json".format(cate1, cate2)
            tmp_df_li = []
            for path in Path(filefolder).rglob(pattern):
                try : 
                    tmp_df = pd.read_json(path) 
                    tmp_df = tmp_df[tmp_df['content'].apply(len) > min_len] #지정 길이 이상의 데이터만 조회
                    if drop_duplicates : #중복 뉴스 삭제
                        tmp_df = tmp_df.drop_duplicates(subset='content', keep='first')
                    tmp_df_li.append(tmp_df)
                except Exception as e:
                    if error_del : os.remove(path) #오류가 있는 파일 삭제. 재작업 목적
                    print("Error file : {}".format(path))
                    print("Error message : {}".format(str(e)))
        
            if len(tmp_df_li) > 1 :                
                print("-- Parttern : {} --".format(pattern)) 
                
                # 세분류별 데이터 건수 제한
                tmp_df = pd.concat(tmp_df_li, sort=False)        
                if maxrow_by_category > 0 : tmp_df = shuffle(tmp_df)[:maxrow_by_category]

                df_li.append(tmp_df)    
    df = pd.concat(df_li, sort=False)
    print("{} rows returned".format(df.shape[0]))
    return df

df = getDataFromFiles("../../NaverNews/navernews/news/news/", category1='정치', category2='*', min_len=200, drop_duplicates=True, maxrow_by_category=10000)

Error file : ../../NaverNews/navernews/news/news/2020/03/02/20200302_정치_정치일반.json
Error message : unhashable type: 'list'
Error file : ../../NaverNews/navernews/news/news/2020/03/01/20200301_정치_정치일반.json
Error message : unhashable type: 'list'
-- Parttern : *_정치_정치일반.json --
-- Parttern : *_정치_국회정당.json --
Error file : ../../NaverNews/navernews/news/news/2019/12/17/20191217_정치_청와대.json
Error message : Unexpected character found when decoding array value (2)
-- Parttern : *_정치_청와대.json --
-- Parttern : *_정치_국방외교.json --
-- Parttern : *_정치_북한.json --
-- Parttern : *_정치_행정.json --
60000 rows returned


In [16]:
df.groupby([df['category1']])['url'].agg(['count'])

,count
category1,
정치,60000


In [17]:
df.groupby([df['category1'], df['category2']])['url'].agg(['count'])

count
category1 category2       
정치        국방/외교      10000
          국회/정당      10000
          북한         10000
          정치일반       10000
          청와대        10000
          행정         10000

In [8]:
import numpy as np 

df_정치[df_정치['summary']!=""].head()

,category1,comment,_type,category2,like,title,url,media,angry,summary,content,warm,updatedate,want,oricategory,date,inputdate,sad
14,정치,1,dict,정치일반,0,"춘천 분구 확정…강원지역 선거구, 원주 빼고 다 바뀐다",https://news.naver.com/main/read.nhn?mode=LS2D...,머니투데이,4,None,김세환 국회의원선거구획정위원회 위원장이 3일 오후 서울 관악구 중앙선거관리위원회 ...,0,2020-03-15 14:10:17,0,정치,2020-03-03,2020.03.03. 오후 6:21,0
17,정치,0,dict,정치일반,0,"홍남기 영남·중부권 감염전문병원 확대, 음압병상도 확충",https://news.naver.com/main/read.nhn?mode=LS2D...,연합뉴스,1,None,관세청 압수 마스크 40만장 풀 것…마스크·방호복 전략물자 지정 검토해야 홍남기 ...,0,2020-03-15 14:10:29,0,정치,2020-03-03,2020.03.03. 오후 6:08,0
18,정치,5240,dict,행정,97,"중국 주민들, 한국 교민 집 각목으로 막고 못질",https://news.naver.com/main/read.nhn?mode=LS2D...,조선일보,14552,None,집에 들어가자 못나오게 만들어 교민의 회사 동료가 각목 제거 중국 안후이성 허페이...,40,2020-03-15 14:10:30,81,정치,2020-03-03,2020.03.03. 오전 1:46,31
21,정치,164,dict,정치일반,34,"이만희, 이번에는 ‘정세균 시계’ 논란..丁 “허무맹랑한 주장”",https://news.naver.com/main/read.nhn?mode=LS2D...,파이낸셜뉴스,264,None,이만희 신천지예수교 증거장막성전 총회장이 2일 기자회견에서 박근혜 전 대통령의 이...,7,2020-03-15 14:10:41,5,정치,2020-03-03,2020.03.03. 오후 5:59,2
22,정치,18,dict,정치일반,63,"[퇴근길] 의사 안철수, 봉사가 '쇼'라고?",https://news.naver.com/main/read.nhn?mode=LS2D...,머니S,15,None,안철수 국민의당 대표 측이 일각에서 제기된 '안 대표는 환자 근처에도 가지 않는다...,0,2020-03-15 14:10:42,2,정치,2020-03-03,2020.03.03. 오후 6:00,2
